# <u> Movie Recommender system : Implicit Ratings </u>

## Introduction

*The aim of this project is to evaluate the performance of user-user Collaborative Filtering, Matrix Factorization and Bayesian Personalized Ranking from scratch and to observe how the results are varying from a naive popularity based recommender system. As opposed to Explicit rating where our aim was to predict the rating, here our aim is to recommend movies to the user.*

### *Implicit Ratings*

Implicit ratings are a type of rating system that are derived from indirect or implicit feedback, rather than explicit ratings or reviews provided by users. Implicit ratings are inferred from a user's behavior or actions, such as the amount of time spent on a particular page, the frequency of clicks or views, or the type of content accessed.

For example, an online retailer may track a customer's browsing and purchasing history to infer their preferences and provide product recommendations based on those implicit ratings. Similarly, a music streaming service may use implicit ratings to create personalized playlists based on a user's listening habits.

### *Dataset*
The MovieLens dataset is a popular benchmark dataset in the field of recommender systems. It contains a large number of movie ratings provided by users, along with movie metadata such as title, genre, and release year. Majorly, the dataset contains three files </br>
* Movie data - which contains details of the movie like Genre, title, release year etc.
* Rating data - which contains the ratings provided by users to each movie. Although the ratings in the dataset are from 0-5, we will map it to a binary rating scale to evaluate implicit recommendation.
* User data - which contains the demographics of users
</br> </br>
For the purpose of this project, we will focus on the rating file and aim to predict the ratings (binary) assigned by each user to individual movies.

### *Algorithms*

Here we will try out the below 3 algorithms and compare their performance through selected evaluation metrics: </br>
* Global average - Most naive form of predicting ratings without taking users into consideration. Here, each user will have the same ratings for a movie which is the average rating of the movie across all users. </br></br>
* User-user collaborative filtering - Here we will find users who have similar tastes and preferences to the user for whom recommendations are being made. Once similar users are identified, a weighted average of their ratings and preferences for movies are used to make recommendations for the target user.</br></br>
* Matrix Factorization -  This algorithm involves breaking down the user-item rating matrix into two lower-dimensional matrices: one representing the users and the other representing the items. These matrices are designed to capture the underlying characteristics of users and items that contribute to their preferences and ratings. These matrices can be used to predict the ratings for new items and make recommendations for users by computing the dot product of the user and item vectors.</br></br>
* Bayesian Personalized Ranking (BPR) - BPR uses a probabilistic model to estimate the likelihood that a user will prefer an item based on their past interactions with the system. Specifically, BPR models the probability that a user prefers one item over another using a pairwise ranking approach. This means that instead of predicting a numerical rating for each item, BPR predicts a preference ranking between pairs of items. </br></br> One of the key advantages of BPR is that it can handle large datasets with sparse interactions, where users have only interacted with a small subset of items. BPR also allows for easy incorporation of additional user and item features, such as demographic information or item attributes, to improve the quality of recommendations.

### *Evaluation Metrics*

We will consider two metrics for evaluation:
* Precision
* Recall

Each metric will be evaluated for top-K recommendations.

## Code

In [23]:
"""
Import required libraries
"""

import pandas as pd
import numpy as np
from numpy.linalg import norm
from scipy.sparse import coo_matrix

from datetime import datetime as dt

### *Reading data*

The file 'ratings.dat' (downloadable from https://grouplens.org/datasets/movielens/1m/) contains data of the format </br>
$<UserID> :: <MovieID> :: <Rating> :: <Timestamp>$
</br>
Let's read and store the data into a Pandas DataFrame

In [10]:
"""
Read data
"""

data_df = pd.read_csv('ratings.dat', sep='::',
                      names=["UserID", "MovieID", "Rating", "Timestamp"],
                      engine='python')
print("Shape of Data : {} x {}\n".format(data_df.shape[0], data_df.shape[1]))
print("SAMPLE FROM ACTUAL DATASET ")
print("--------------------------")
print(data_df.head())

TRAIN_SIZE = 0.7


# First, generate dictionaries for mapping old id to new id for users
# and movies
# unique_MovieID = data_df['MovieID'].unique()
unique_UserID = data_df['UserID'].unique()
j = 0
user_old2new_id_dict = dict()
for u in unique_UserID:
    user_old2new_id_dict[u] = j
    j += 1
j = 0
movie_old2new_id_dict = dict()
for i in unique_MovieID:
    movie_old2new_id_dict[i] = j
    j += 1

# Then, use the generated dictionaries to reindex UserID and MovieID
# in the data_df
user_list = data_df['UserID'].values
movie_list = data_df['MovieID'].values
for j in range(len(data_df)):
    user_list[j] = user_old2new_id_dict[user_list[j]]
    movie_list[j] = movie_old2new_id_dict[movie_list[j]]
data_df['UserID'] = user_list
data_df['movieID'] = movie_list

# Generate train_df with 70% samples and test_df with 30% samples,
# and there should have no overlap between them.
train_index = np.random.random(len(data_df)) <= TRAIN_SIZE
train_df = data_df[train_index]
test_df = data_df[~train_index]

# Number of Unique Users
num_user = len(data_df['UserID'].unique())   
# Number of Unique Movies
num_movie = len(data_df['MovieID'].unique())

# Generate train_mat and test_mat
train_mat = coo_matrix(
    (train_df['Rating'].values,
     (train_df['UserID'].values, train_df['MovieID'].values)),
    shape=(num_user, num_movie)
).astype(float).toarray()
test_mat = coo_matrix(
    (test_df['Rating'].values,
     (test_df['UserID'].values, test_df['MovieID'].values)),
    shape=(num_user, num_movie)
).astype(float).toarray()

# Visualize the new Matrix : rows -> users, columns -> movies
print("\n\nDATA CONVERTED TO USER x MOVIES FORMAT")
print("--------------------------------------")
print(pd.DataFrame(train_mat, dtype = int))

Shape of Data : 1000209 x 4

SAMPLE FROM ACTUAL DATASET 
--------------------------
   UserID  MovieID  Rating  Timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408       4  978300275
4       1     2355       5  978824291


DATA CONVERTED TO USER x MOVIES FORMAT
--------------------------------------
      0     1     2     3     4     5     6     7     8     9     ...  3696  \
0        0     3     0     4     5     3     0     0     0     4  ...     0   
1        0     0     0     0     0     0     0     0     0     0  ...     0   
2        0     0     0     0     5     5     0     0     0     0  ...     0   
3        0     0     0     0     0     0     0     0     0     0  ...     0   
4        0     0     0     3     5     0     0     0     0     4  ...     0   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
6035     5     0     3     0     0     5     0     0   

We will convert explicit recommendations to implicit by replacing all non-zero ratings as 1. The idea is that whenever a user 'rated' the movie, he/she liked it (although it is not always correct)

In [12]:
# turn the explicit ratings to implicit feedback data
train_mat = (train_mat > 0).astype(float)
test_mat = (test_mat > 0).astype(float)

# Visualize the new Matrix : rows -> users, columns -> movies
print("\n\nDATA CONVERTED TO IMPLICIT RATINGS")
print("--------------------------------------")
print(pd.DataFrame(train_mat, dtype = int))



DATA CONVERTED TO IMPLICIT RATINGS
--------------------------------------
      0     1     2     3     4     5     6     7     8     9     ...  3696  \
0        0     1     0     1     1     1     0     0     0     1  ...     0   
1        0     0     0     0     0     0     0     0     0     0  ...     0   
2        0     0     0     0     1     1     0     0     0     0  ...     0   
3        0     0     0     0     0     0     0     0     0     0  ...     0   
4        0     0     0     1     1     0     0     0     0     1  ...     0   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
6035     1     0     1     0     0     1     0     0     1     0  ...     0   
6036     1     0     0     0     0     0     0     1     0     1  ...     0   
6037     0     0     0     0     0     0     0     0     0     0  ...     0   
6038     0     1     1     0     0     1     0     0     0     1  ...     0   
6039     1     0     0     0     0     0     0     0   

### *Evaluation metrics*

As mentioned earlier, we will implement Precision and Recall metrics to evaluate our recommender models. 

</br>

* $ Precision = \frac{Relevant \cap Recommended}{Relevant}$

* $ Recall = \frac{Relevant \cap Recommended}{Recommended}$

In [16]:
def precision_n_recall_at_k(recommendations, actual_likes, k):
    
    if actual_likes.sum() == 0:
        return (np.NaN, np.NaN)
    
    true_positive = 0
    for i in range(k):
        if actual_likes[int(recommendations[i])] == 1:
            true_positive +=1
            
    precision = true_positive / k
    recall = true_positive / actual_likes.sum()
    
    return (precision, recall)

In [19]:
def evaluate(recommended_movies):
    
    precision_at_5 = []
    precision_at_20 = []
    precision_at_50 = []

    recall_at_5 = []
    recall_at_20 = []
    recall_at_50 = []

    for i in range(test_mat.shape[0]):
        # Precision and recall at 5
        p_r_5 =  precision_n_recall_at_k(recommendations = recommended_movies[i],
                                         actual_likes = test_mat[i], k = 5)
        precision_at_5.append(p_r_5[0])
        recall_at_5.append(p_r_5[1])

        # Precision and recall at 20
        p_r_20 =  precision_n_recall_at_k(recommendations = recommended_movies[i],
                                         actual_likes = test_mat[i], k = 20)
        precision_at_20.append(p_r_20[0])
        recall_at_20.append(p_r_20[1])

        # Precision and recall at 50
        p_r_50 =  precision_n_recall_at_k(recommendations = recommended_movies[i],
                                         actual_likes = test_mat[i], k = 50)
        precision_at_50.append(p_r_50[0])
        recall_at_50.append(p_r_50[1])

    print("Avg. Precision at 05 : {:.5f}".format(np.array(precision_at_5).mean()))
    print("Avg. Precision at 20 : {:.5f}".format(np.array(precision_at_20).mean()))
    print("Avg. Precision at 50 : {:.5f}".format(np.array(precision_at_50).mean()))
    print("")
    print("Avg. Recall at 05 : {:.5f}".format(np.array(recall_at_5).mean()))
    print("Avg. Recall at 20 : {:.5f}".format(np.array(recall_at_20).mean()))
    print("Avg. Recall at 50 : {:.5f}".format(np.array(recall_at_50).mean()))

------

### 1. Global Average Rating a.k.a. Popular Movie recommender ('Non-personalized')

Here we will suggest movies based on their overall popularity, meaning those that have received high ratings from a larger number of users will be recommended to all. Therefore, it is a very naive approach and personalization is not a factor in this approach.
</br></br>
To determine a movie's popularity, we will calculate its mean rating.

In [21]:
# Implementation of Popularity based recommender

total_implcit_ratings = train_mat.sum(axis = 0)
popular_movies = np.argsort(-total_implcit_ratings)
recommended_movies = np.zeros((train_mat.shape[0], 50))

for user_ind, user_ratings in enumerate(train_mat):
    watched_movies = np.nonzero(user_ratings)[0]
    counter = 0

    recommendation = []
    for movie in popular_movies:
        if (counter < 50) and (movie not in watched_movies):
            recommendation.append(movie)
            counter +=1
    recommended_movies[user_ind] = recommendation

In [22]:
evaluate(recommended_movies)

Avg. Precision at 05 : 0.26682
Avg. Precision at 20 : 0.20911
Avg. Precision at 50 : 0.15792

Avg. Recall at 05 : 0.03630
Avg. Recall at 20 : 0.10818
Avg. Recall at 50 : 0.19471


------

### 2. User-User Collaborative Filtering

User-User Collaborative Filtering is a type of recommendation system that is based on the idea that people who have similar preferences in the past are likely to have similar preferences in the future. This technique uses the behavior and choices of other users to make recommendations to a target user.
</br></br>
To apply user-user collaborative filtering, the system identifies a target user and then looks for other users with similar tastes and preferences. It then analyzes the ratings and preferences of those similar users to recommend items that the target user has not yet seen or rated. The system can make use of various similarity metrics, such as Pearson correlation or cosine similarity, to identify the most similar users.
</br></br>
This technique has several advantages over other recommendation approaches, such as content-based filtering. User-user collaborative filtering can identify and recommend items that may be outside a user's typical preferences, but are enjoyed by similar users. It also doesn't rely on metadata or content information about items, which may be incomplete or inaccurate. However, this approach may face scalability issues with larger user bases and may require frequent updates to keep up with changing user preferences.
</br></br>

In [24]:
# Cosine Calculation
def cosine(vec1, vec2, user1_ind, user2_ind):  
    return np.dot(vec1,vec2) / (user_norm[user1_ind] * user_norm[user2_ind])

user_user_similarity = np.zeros((train_mat.shape[0], train_mat.shape[0]))
user_norm = np.zeros(train_mat.shape[0])

# Calculate L2 norm of each user vector 
# beforehand to reduce calculation time
for ind, row in enumerate(train_mat):
    user_norm[ind] = norm(row)
    
# For each user-user pair calculate cosine similarity
start = dt.now()
for user1_ind, vec1 in enumerate(train_mat):
    user2_ind = user1_ind +1
    for vec2 in train_mat[user1_ind+1:, :]:
        user_user_similarity[user1_ind][user2_ind] = cosine(vec1, vec2, user1_ind, user2_ind)
        user2_ind += 1

# Set diagonal cosine values as 0 
user_user_similarity = user_user_similarity + user_user_similarity.T
end = dt.now()
print("Time taken : {}".format((end-start).__str__()))

# Identify top N similar users for each user
N = 10
top_N_similar_users = np.zeros((train_mat.shape[0], 10), dtype = int)
for user_ind, user in enumerate(user_user_similarity):
    top_N_similar_users[user_ind] = (-user).argsort()[:N]
    

# Using similarity scores from each user, predict movie ratings for each user
cf_recommendations = np.zeros((train_mat.shape[0], train_mat.shape[1]))
top_50_recommendations = np.zeros((train_mat.shape[0], 50), dtype = int)

for user_ind, user in enumerate(user_user_similarity):
    movie_ratings = np.zeros(train_mat.shape[1])
    for similar_user in top_N_similar_users[user_ind]:
        movie_ratings += user_user_similarity[user_ind,int(similar_user)] * train_mat[similar_user, :]
    movie_ratings = movie_ratings / norm([user_user_similarity[i] 
                                          for i in top_N_similar_users[user_ind]], ord = 1)
    cf_recommendations[user_ind] = movie_ratings 
    
# Identify top-50 movies for each user
for user_ind in range(train_mat.shape[0]):
    watched_movies = np.nonzero(train_mat[user_ind])[0]
    recommendation = []
    counter = 0
    for movie in np.argsort(-cf_recommendations[user_ind]):
        if (counter < 50) and (movie not in watched_movies):
            recommendation.append(movie)
            counter +=1
    top_50_recommendations[user_ind] = recommendation

Time taken : 0:00:41.936703


In [25]:
evaluate(top_50_recommendations)

Avg. Precision at 05 : 0.37930
Avg. Precision at 20 : 0.29185
Avg. Precision at 50 : 0.22187

Avg. Recall at 05 : 0.07083
Avg. Recall at 20 : 0.19116
Avg. Recall at 50 : 0.32168


-----

### 3. Matrix Factorization

Matrix Factorization is a technique used in recommendation systems to model the preferences of users and items in a more efficient way. The idea behind this technique is to factorize a large user-item interaction matrix into two smaller matrices: a user matrix and an item matrix. The user matrix represents the preferences of each user, while the item matrix represents the attributes of each item. </br></br>

The factorization process can be achieved using various algorithms, such as Singular Value Decomposition (SVD), Non-Negative Matrix Factorization (NMF), and Alternating Least Squares (ALS). The effectiveness of matrix factorization depends on the quality of the data, the selection of appropriate algorithm and hyperparameters, and the evaluation metrics used to measure the performance of the system.

The MF model can be mathematically represented as: 

<center>$\underset{\mathbf{P},\mathbf{Q}}{\text{min}}\,\,L=\sum_{(u,i)\in\mathcal{O}}(\mathbf{P}_u\cdot\mathbf{Q}^\top_i-r_{u,i})^2+\lambda(\lVert\mathbf{P}\rVert^2_{\text{F}}+\lVert\mathbf{Q}\rVert^2_{\text{F}})$,</center>
    
where $\mathbf{P}$ is the user latent factor matrix of size (#user, #latent); $\mathbf{Q}$ is the movie latent factor matrix of size (#movie, #latent); $\mathcal{O}$ is a user-movie pair set containing all user-movie pairs having ratings in train_mat; $r_{u,i}$ represents the rating for user u and movie i; $\lambda(\lVert\mathbf{P}\rVert^2_{\text{F}}+\lVert\mathbf{Q}\rVert^2_{\text{F}})$ is the regularization term to overcome overfitting problem, $\lambda$ is the regularization weight (a hyper-parameter manually set by developer, i.e., you), and $\lVert\mathbf{P}\rVert^2_{\text{F}}=\sum_{x}\sum_{y}(\mathbf{P}_{x,y})^2$, $\lVert\mathbf{Q}\rVert^2_{\text{F}}=\sum_{x}\sum_{y}(\mathbf{Q}_{x,y})^2$. Such an L function is called the **loss function** for the matrix factorization model. The goal of training an MF model is to find appropriate $\mathbf{P}$ and $\mathbf{Q}$ to minimize the loss L.

The main idea is the same as the MF model for explicit ratings. But instead of predicting explicit ratings, here the MF is to predict binary ratings based on which movies are ranked for users.

The challenge now is that in an implicit feedback dataset, there is only positive signal (i.e., '1' in the train_mat) without negative signal. Hence, to let our MF work for this implicit feedback, a simple but powerful method -- random negative sampling -- is adopted. The main idea is that in each training epoch, we randomly sample user-movie pairs without positive feedback in train_mat (user-movie pairs with '0' in train_mat) to be negative feedback, and mix them with positive feedback as the training data to train the MF model. 

In [26]:
class MF_implicit:
    def __init__(self, train_mat, test_mat, latent=5, lr=0.01, reg=0.01):
        self.train_mat = train_mat  # the training rating matrix of size (#user, #movie)
        self.test_mat = test_mat  # the training rating matrix of size (#user, #movie)
        
        self.latent = latent  # the latent dimension
        self.lr = lr  # learning rate
        self.reg = reg  # regularization weight, i.e., the lambda in the objective function
        
        self.num_user, self.num_movie = train_mat.shape
        
        self.sample_user, self.sample_movie = self.train_mat.nonzero()  # get the user-movie paris having ratings in train_mat
        self.num_sample = len(self.sample_user)  # the number of user-movie pairs having ratings in train_mat

        self.user_test_like = []
        for u in range(self.num_user):
            self.user_test_like.append(np.where(self.test_mat[u, :] > 0)[0])

        self.P = np.random.random((self.num_user, self.latent))  # latent factors for users, size (#user, self.latent), randomly initialized
        self.Q = np.random.random((self.num_movie, self.latent))  # latent factors for users, size (#movie, self.latent), randomly initialized
        
    def negative_sampling(self):
        negative_movie = np.random.choice(np.arange(self.num_movie), size=(len(self.sample_user)), replace=True)
        true_negative = self.train_mat[self.sample_user, negative_movie] == 0
        negative_user = self.sample_user[true_negative]
        negative_movie = negative_movie[true_negative]
        return np.concatenate([self.sample_user, negative_user]), np.concatenate([self.sample_movie, negative_movie])

    def train(self, epoch=20):

        
        for ep in range(epoch):
           
            # Call negative_sampling to get data
            users, movies = self.negative_sampling()
            
            # Shuffle the data
            p = np.random.permutation(len(users))
            users, movies = users[p], movies[p]
            
            # Iterate over each user-movie pair and train
            for user, movie in zip(users, movies):
                r_ui = self.train_mat[user, movie]
                self.P[user, :] -= self.lr * (2* (np.dot(self.P[user, :],self.Q[movie, :]) - r_ui) *
                                              self.Q[movie, :] + 2 * self.reg * self.P[user, :] )
                self.Q[movie, :] -= self.lr * (2* (np.dot(self.P[user, :],self.Q[movie, :]) - r_ui) *
                                              self.P[user, :] + 2 * self.reg * self.Q[movie, :] )
                               

        # Test the model
        self.test()
        
            
    def predict(self):

        predictions = np.matmul(self.P,  self.Q.T )
        recommendation = np.zeros((self.train_mat.shape[0], 50), dtype = int)
        
        for user_ind in range(self.train_mat.shape[0]):
            watched_movies = np.nonzero(self.train_mat[user_ind])[0]
            recommendation_user = []
            counter = 0
            for movie in np.argsort(-predictions[user_ind]):
                if (counter < 50) and (movie not in watched_movies):
                    recommendation_user.append(movie)
                    counter +=1
            recommendation[user_ind] = recommendation_user
        
        return recommendation
    
    def test(self):
        recommendation = self.predict()

        recalls = np.zeros(3)
        precisions = np.zeros(3)
        user_count = 0.

        for u in range(self.num_user):
            test_like = self.user_test_like[u]
            test_like_num = len(test_like)
            if test_like_num == 0:
                continue
            rec = recommendation[u, :]
            hits = np.zeros(3)
            for k in range(50):
                if rec[k] in test_like:
                    if k < 50:
                        hits[2] += 1
                        if k < 20:
                            hits[1] += 1
                            if k < 5:
                                hits[0] += 1
            recalls[0] += (hits[0] / test_like_num)
            recalls[1] += (hits[1] / test_like_num)
            recalls[2] += (hits[2] / test_like_num)
            precisions[0] += (hits[0] / 5.)
            precisions[1] += (hits[1] / 20.)
            precisions[2] += (hits[2] / 50.)
            user_count += 1

        recalls /= user_count
        precisions /= user_count

        print('recall@5\t[%.6f],\t||\t recall@20\t[%.6f],\t||\t recall@50\t[%.6f]' % (recalls[0], recalls[1], recalls[2]))
        print('precision@5\t[%.6f],\t||\t precision@20\t[%.6f],\t||\t precision@50\t[%.6f]' % (precisions[0], precisions[1], precisions[2]))
        print('')

Now, run the next cell to build and train your implemented MF model for implicit feedback. The expected time used for training one epoch is 20s to 2 min.

In [27]:
mf_implicit = MF_implicit(train_mat, test_mat, latent=5, lr=0.01, reg=0.0001)
mf_implicit.train(epoch=20)

recall@5	[0.039198],	||	 recall@20	[0.123433],	||	 recall@50	[0.238562]
precision@5	[0.293212],	||	 precision@20	[0.243071],	||	 precision@50	[0.197904]



Although, the performance of Matrix Factorization is slightly better than non-personalized recommender system, Matrix Factorization (MF) is not providing a better result that User-user collaborative filtering (UUCF). <br>

MF tries to find the characteristics of Users and Movies through its latent factors and hence it has a personalized touch to it. So, it is better than a non-personlaized recommeder system.  <br>

The poor performance of MF may be because of the implicit nature of dataset. Here we are giving incorrect information to the model by saying that a user movie pair with no feedback implies it is a negative feedback. Hence, the latent features learned by the model can be inaccurate leading to poor performance.

------------------

### 4. Bayesian Personalized Ranking (BPR)

The BPR model can be mathematically represented as: 

<center>$\underset{\mathbf{P},\mathbf{Q}}{\text{max}}\,\,L=\sum_{(u,i,j)\in\mathcal{O}}\sigma(\mathbf{P}_u\cdot\mathbf{Q}^\top_i-\mathbf{P}_u\cdot\mathbf{Q}^\top_j)+\lambda(\lVert\mathbf{P}\rVert^2_{\text{F}}+\lVert\mathbf{Q}\rVert^2_{\text{F}})$,</center>
    
where $\mathbf{P}$ is the user latent factor matrix of size (#user, #latent); $\mathbf{Q}$ is the movie latent factor matrix of size (#movie, #latent); $\sigma(\cdot)$ is the Sigmoid function; $\mathcal{O}$ is a (user, positive movie, negative movie) tuple set, and each tuple $(u,i,j)$ is a training sample with user $u$ and a posotive movie $i$ with value 1 in train_mat and a negative movie $j$ with value 0 in train_mat; $\lambda(\lVert\mathbf{P}\rVert^2_{\text{F}}+\lVert\mathbf{Q}\rVert^2_{\text{F}})$ is the regularization term to overcome overfitting problem, $\lambda$ is the regularization weight (a hyper-parameter manually set by the developer, i.e., you), and $\lVert\mathbf{P}\rVert^2_{\text{F}}=\sum_{x}\sum_{y}(\mathbf{P}_{x,y})^2$, $\lVert\mathbf{Q}\rVert^2_{\text{F}}=\sum_{x}\sum_{y}(\mathbf{Q}_{x,y})^2$. Such an L function is called the **loss function** for the matrix factorization model. The goal of training a BPR model is to find appropriate $\mathbf{P}$ and $\mathbf{Q}$ to **maximize** the loss L.

In [28]:
class BPR:
    def __init__(self, train_mat, test_mat, latent=5, lr=0.01, reg=0.01):
        self.train_mat = train_mat  # the training rating matrix of size (#user, #movie)
        self.test_mat = test_mat  # the training rating matrix of size (#user, #movie)
        
        self.latent = latent  # the latent dimension
        self.lr = lr  # learning rate
        self.reg = reg  # regularization weight, i.e., the lambda in the objective function
        
        self.num_user, self.num_movie = train_mat.shape
        
        self.positive_user, self.positive_movie = self.train_mat.nonzero()  # get the user-movie paris having ratings in train_mat

        self.user_test_like = []
        for u in range(self.num_user):
            self.user_test_like.append(np.where(self.test_mat[u, :] > 0)[0])

        self.P = np.random.random((self.num_user, self.latent))  # latent factors for users, size (#user, self.latent), randomly initialized
        self.Q = np.random.random((self.num_movie, self.latent))  # latent factors for users, size (#movie, self.latent), randomly initialized
        
    def negative_sampling(self): 
        # do the negative sampling for each of the positive user-movie pair. Here we set negative sampling rate as 2, 
        # i.e., for each positive user-movie pair, randomly sample two negative movies. 
        # This function returns final training data: a user list, a positive movie list, and a negative movie list
        sample_user = np.tile(self.positive_user, 2)
        sample_pos_movie = np.tile(self.positive_movie, 2)
        sample_neg_movie = np.random.choice(np.arange(self.num_movie), size=(len(sample_user)), replace=True)
        true_negative = self.train_mat[sample_user, sample_neg_movie] == 0
        return sample_user[true_negative], sample_pos_movie[true_negative], sample_neg_movie[true_negative]

    def train(self, epoch=20):

        for ep in range(epoch):

            
            # Call negative_sampling to get data
            users, pos_movies, neg_movies = self.negative_sampling()
            
            # Shuffle the data
            p = np.random.permutation(len(users))
            users, pos_movies, neg_movies = users[p], pos_movies[p], neg_movies[p] 
            
            # Iterate over each user-movie pair and train
            for user, pos_movie, neg_movie in zip(users, pos_movies, neg_movies):
                x_hat = np.matmul(self.P[user, :], self.Q[pos_movie, :]) - \
                        np.matmul(self.P[user, :], self.Q[neg_movie, :])
                self.P[user, :] -= self.lr * ( - np.exp(-x_hat)/(1 +np.exp(-x_hat)) * 
                                              (self.Q[pos_movie, :] - self.Q[neg_movie, :]) +
                                              self.reg * self.P[user, :])
                self.Q[pos_movie, :] -= self.lr * ( - np.exp(-x_hat)/(1 +np.exp(-x_hat)) * 
                                              self.P[user, :] + self.reg * self.Q[pos_movie, :])
                self.Q[neg_movie, :] -= self.lr * (np.exp(-x_hat)/(1 +np.exp(-x_hat)) * 
                                              self.P[user, :] + self.reg * self.Q[neg_movie, :])
                  
            
        # Test the model
        self.test()

            
    def predict(self):
        # The prediction function, which generates the ranked lists of movies 
        # by the trained BPR for every user, store the result (named 'recommendation') in a numpy array of size (#user, 50), where entry (u, k) 
        # represents the movie id that is ranked at position k in the recommendation list to user u. Return the 'recommendation' variable. 
        prediction_mat = np.matmul(self.P, self.Q.T)
        recommendation = []
        for u in range(self.num_user):
            scores = prediction_mat[u]
            train_like = np.where(train_mat[u, :] > 0)[0]
            scores[train_like] = -9999
            top50_iid = np.argpartition(scores, -50)[-50:]
            top50_iid = top50_iid[np.argsort(scores[top50_iid])[-1::-1]]
            recommendation.append(top50_iid)
        recommendation = np.array(recommendation)
        return recommendation
    
    def test(self):
        recommendation = self.predict()

        recalls = np.zeros(3)
        precisions = np.zeros(3)
        user_count = 0.

        for u in range(self.num_user):
            test_like = self.user_test_like[u]
            test_like_num = len(test_like)
            if test_like_num == 0:
                continue
            rec = recommendation[u, :]
            hits = np.zeros(3)
            for k in range(50):
                if rec[k] in test_like:
                    if k < 50:
                        hits[2] += 1
                        if k < 20:
                            hits[1] += 1
                            if k < 5:
                                hits[0] += 1
            recalls[0] += (hits[0] / test_like_num)
            recalls[1] += (hits[1] / test_like_num)
            recalls[2] += (hits[2] / test_like_num)
            precisions[0] += (hits[0] / 5.)
            precisions[1] += (hits[1] / 20.)
            precisions[2] += (hits[2] / 50.)
            user_count += 1

        recalls /= user_count
        precisions /= user_count

        print('recall@5\t[%.6f],\t||\t recall@20\t[%.6f],\t||\t recall@50\t[%.6f]' % (recalls[0], recalls[1], recalls[2]))
        print('precision@5\t[%.6f],\t||\t precision@20\t[%.6f],\t||\t precision@50\t[%.6f]' % (precisions[0], precisions[1], precisions[2]))
        print('')

In [29]:
bpr = BPR(train_mat, test_mat, latent=5, lr=0.01, reg=0.001)
bpr.train(epoch=20)

recall@5	[0.050495],	||	 recall@20	[0.141984],	||	 recall@50	[0.261247]
precision@5	[0.361854],	||	 precision@20	[0.277268],	||	 precision@50	[0.215030]



BPR is providing much better result than non-personalized recommender system and Matrix factorization (MF) and a similar result as that of User-user collaborative filtering (UUCF). <br>

It performs better than non-personalized recommenders as it is a personalized recommender system that can capture user preferences and movie characteristics. BPR is expected to perform better than MF as BPR is designed for implicit ratings (where an absence of rating doesn't mean its negatively rated). Moreover, BPR directly optimizes the ranking of items, which is a more natural and intuitive objective for recommendation systems than predicting explicit ratings. 

------
------

## THE END